In [1]:
import imp
import os
import cv2
import pdb
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm
from multiprocessing import Pool
import torch
from torch import optim
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
# from torch.nn.functional import InterpolationMode
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
import logging 
import datetime
import sys
import cv2
import matplotlib.pyplot as plt
import PIL

In [2]:
gloss_df_path = "data_validation/processed_gloss.csv"
gloss_df = pd.read_csv(gloss_df_path)
gloss_df.dropna(inplace=True)
gloss_df.replace(to_replace="ASHAG", value="AŞAĞI", inplace=True)
gloss_df['glossRange'] = gloss_df['glossEnd'] - gloss_df['glossStart']
# gloss_df.head()

In [3]:
classes = sorted(gloss_df.gloss.unique())
num_classes = len(classes)

In [4]:
class_to_idx = {classes[i]: i for i in range(len(classes))}
idx_to_class = {i: classes[i] for i in range(len(classes))}

In [5]:
import torchviz
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from torchvision import transforms

In [6]:
from pytorchvideo.data import labeled_video_dataset

from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute,   
)

from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomAdjustSharpness,
    Resize,
    RandomHorizontalFlip
)

from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:25: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in 0.14. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [19]:
video_transform = Compose([
    ApplyTransformToKey(key="video",
    transform=Compose([
        UniformTemporalSubsample(25),
        Lambda(lambda x: x/255),
        Normalize((0.45, 0.45, 0.45), (0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=256, max_size=512),
        CenterCropVideo(256),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [8]:
train_path = "/home/toghrul/SLR/data/binary-data/train"
val_path = "/home/toghrul/SLR/data/binary-data/val"
test_path = "/home/toghrul/SLR/data/binary-data/test"

In [9]:
bin_classes = []

for gloss in os.listdir(test_path):
    bin_classes.append(gloss)

In [10]:
bin_classes = sorted(bin_classes)
bin_class_to_idx = {bin_classes[i]: i for i in range(len(bin_classes))}
bin_idx_to_class = {i: bin_classes[i] for i in range(len(bin_classes))}
bin_idx_to_class

{0: 'MƏN', 1: 'VAR'}

In [11]:
from pytorch_lightning import LightningModule, seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from torch.optim.lr_scheduler import CosineAnnealingLR, ReduceLROnPlateau, CosineAnnealingWarmRestarts
from sklearn.metrics import classification_report
import torchmetrics

In [10]:
video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [21]:
dataset = labeled_video_dataset(train_path, transform=video_transform, clip_sampler=make_clip_sampler('random', 2), decode_audio=False)

loader = DataLoader(dataset, batch_size=8, num_workers=0, pin_memory=True)

In [22]:
batch = next(iter(loader))

In [29]:
batch.keys()

dict_keys(['video', 'video_name', 'video_index', 'clip_index', 'aug_index', 'label'])

In [33]:
batch["label"]

tensor([0, 0, 0, 0, 1, 0, 0, 0])

In [ ]:
b

In [11]:
# a = torch.randn(size=(8, num_classes)).requires_grad_()
# b = torch.randint(5, size=(8, ), dtype=torch.long)
# a, b

In [12]:
# b = F.one_hot(b, num_classes=num_classes).float()

In [13]:
# criterion = nn.BCEWithLogitsLoss()
# criterion(a, b)

In [14]:
# loss = nn.CrossEntropyLoss()
# input = torch.randn(3, 5, requires_grad=True)
# target = torch.empty(3, dtype=torch.long).random_(5)
# output = loss(input, target)
# output

In [15]:
# input, target

In [18]:
class VideoModel(LightningModule):
    def __init__(self, ):
        super(VideoModel, self).__init__()

        self.video_model = torch.hub.load('facebookresearch/pytorchvideo', 'efficient_x3d_xs', pretrained=True)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(400, 1)

        self.lr = 1e-3
        self.batch_size = 8
        self.num_worker = 4
        self.num_steps_train = 0
        self.num_steps_val = 0

        # self.metric = torchmetrics.classification.MultilabelAccuracy(num_labels=num_classes)
        self.metric = torchmetrics.Accuracy()
        
        #loss
        self.criterion = nn.BCEWithLogitsLoss()

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.fc(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr = self.lr)
        scheduler = ReduceLROnPlateau(opt, mode="min", factor=0.05, patience=2, min_lr=1e-6)
        # scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt,
                'lr_scheduler': scheduler, 
                "monitor": "val_loss"}

    def train_dataloader(self):
        dataset = labeled_video_dataset(train_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f"Label: {label}\nPred: {out}")
        print(f">>> Training step No.{self.num_steps_train}:")
        # print("Pred:", out)
        # print("GT:", label)
        # print(f"Pred:\n{out}")
        # print(f"Pred shape:\n{out.shape}")
        # print(f"Label:\n{label}")
        # print(f"Label shape:\n{label.shape}")
        # print(">>> INFO: Computing Training Loss")
        
        loss = self.criterion(out, label.float().unsqueeze(1))
        
        print(f"Loss: {loss}")
        self.num_steps_train += 1
        # print(">>> INFO: Training Loss Computed")
        # print(">>> INFO: Computing Training Metric")
        # metric = self.metric(out, label)
        
        # Below is for Accuracy
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
        
        print(f"Accuracy: {metric}")

        values = {"loss": loss,
                "metric": metric.detach()}
        
        self.log_dict({"step_loss": loss,
                        "step_metric": metric.detach()})
        
        return values
        
        # return {"loss": loss}

    def training_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        
        self.log('training_loss', loss)
        print(f">>> Epoch end loss: {loss}")
        self.log('training_metric', metric)
        

    def val_dataloader(self):
        dataset = labeled_video_dataset(val_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self(video)
        print(f"Label: {label}\nPred: {out}")
        # print(">>> INFO: Computing Val Loss")
        print(f">>> Validation step No.{self.num_steps_val}:")

        loss = self.criterion(out, label.float().unsqueeze(1))
        print()
        print(f"Loss: {loss}")
        self.num_steps_val += 1
        # print(">>> INFO: Val Loss Computed")
        # print(">>> INFO: Computing Val Metric")
        # metric = self.metric(out, label)
        # Below is for Accuracy
        
        metric = self.metric(out, label.to(torch.int64).unsqueeze(1))
                
        print(f"Accuracy: {metric}")
        

        return {"loss": loss,
                "metric": metric.detach()}
        
        # return {"loss": loss}

    def validation_epoch_end(self, outputs):
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().numpy().round(2)
        self.log('val_loss', loss)
        self.log('val_metric', metric)

    def test_dataloader(self):
        dataset = labeled_video_dataset(test_path, clip_sampler=make_clip_sampler('random', 2),
                                         transform=video_transform, decode_audio=False)

        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.num_worker, pin_memory=True)

        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        # label = F.one_hot(label, num_classes=num_classes).float()
        out = self.forward(video)
        # metric = self.metric(out, label)

        return {"label": label,
                "pred": out.detach(),}

    def test_epoch_end(self, outputs):
        label=torch.cat([x['label'] for x in outputs]).cpu().numpy()
        pred = torch.cat([x['pred'] for x in outputs]).cpu().numpy()
        # self.log('test_loss', loss)
        # self.log('test_metric', metric)
        
        # Below is for MultiLabelClassification
        # class_labels = label.argmax(axis=1)
        # class_pred = pred.argmax(axis=1)
        
        
        # Below is for BinaryClassification
        class_labels = label
        class_pred = np.where(pred > 0, 1, 0)
        
        print(f">> Label: {class_labels}")
        print(f">> Pred: {class_pred.squeeze()}")
              
        print(classification_report(class_labels, class_pred))
        
        return {"prediction": class_pred,
                "labels": class_labels}




In [35]:
checkpoint_callback = ModelCheckpoint(monitor='val_loss', dirpath="checkpoints", 
                                    verbose=True, save_last=True, save_top_k=2)
lr_monitor = LearningRateMonitor(logging_interval="epoch")


In [40]:
model = VideoModel()
seed_everything(0)

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main
Global seed set to 0


0

In [36]:
trainer = Trainer(max_epochs=15,
                accelerator="gpu", devices=-1,
                precision=16,
                # accumulate_grad_batches=2,
                enable_progress_bar=True,
                # num_sanity_val_steps=0,
                callbacks=[lr_monitor, checkpoint_callback],
                log_every_n_steps=5,
                limit_train_batches=25,
                limit_val_batches=10,
                limit_test_batches=10,)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [19]:
# trainer.fit(model, ckpt_path="/home/toghrul/SLR/sign-lang/checkpoints/last.ckpt")
trainer.fit(model)

/home/toghrul/anaconda3/envs/sign-lang/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:616: UserWarning: Checkpoint directory /home/toghrul/SLR/sign-lang/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type              | Params
--------------------------------------------------
0 | video_model | EfficientX3d      | 3.8 M 
1 | relu        | ReLU              | 0     
2 | fc          | Linear            | 401   
3 | metric      | Accuracy          | 0     
4 | criterion   | BCEWithLogitsLoss | 0     
--------------------------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
7.589     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-1.0410],
        [-0.6724],
        [-0.9170],
        [-0.9272],
        [-0.9751],
        [-0.2600],
        [-0.8491],
        [-0.7397]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.0:

Loss: 0.3777940571308136
Accuracy: 1.0
Label: tensor([0, 1, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-0.6021],
        [-0.5103],
        [-0.9604],
        [-0.8262],
        [-0.2499],
        [-0.5225],
        [-0.9497],
        [-0.6992]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.1:

Loss: 0.48455989360809326
Accuracy: 0.875


Training: 0it [00:00, ?it/s]

Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-0.5894],
        [-0.1630],
        [-0.5737],
        [-0.2391],
        [-0.0739],
        [-0.0850],
        [-0.0891],
        [-0.4849]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)
>>> Training step No.0:
Loss: 0.5652141571044922
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-1.9570],
        [-1.7227],
        [-3.8359],
        [-2.1289],
        [-3.0488],
        [-3.3555],
        [-3.4355],
        [-2.3027]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)
>>> Training step No.1:
Loss: 0.07973060756921768
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 1, 1, 0, 0], device='cuda:0')
Pred: tensor([[-7.4609],
        [-6.0664],
        [-7.1484],
        [-6.7734],
        [-6.4102],
        [-5.2305],
        [-8.2969],
        [-5.4492]], device='cuda:0', dtype=torch.float16,
       grad_fn=<AddmmBackward0>)
>>> Training

...

Validation: 0it [00:00, ?it/s]

Label: tensor([1, 1, 0, 0, 0, 0, 0, 1], device='cuda:0')
Pred: tensor([[  3.0391],
        [  1.5684],
        [ -7.3086],
        [ -3.6973],
        [-11.4688],
        [ -6.6641],
        [-18.3906],
        [  5.9609]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.142:

Loss: 0.03313415125012398
Accuracy: 1.0
Label: tensor([1, 0, 0, 0, 1, 0, 0, 1], device='cuda:0')
Pred: tensor([[  7.4023],
        [-12.0312],
        [-10.0391],
        [ -4.4844],
        [  5.3867],
        [ -7.4375],
        [-12.6562],
        [  7.3281]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.143:

Loss: 0.0022118762135505676
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[ -7.3633],
        [-10.5781],
        [ -5.0859],
        [ -3.3164],
        [ -6.9375],
        [-13.8750],
        [ -8.3125],
        [-10.1406]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.144:

Loss: 0.00546498317271471
Accuracy: 1.0
Label

Epoch 14, global step 375: 'val_loss' reached 0.01000 (best 0.01000), saving model to '/home/toghrul/SLR/sign-lang/checkpoints/epoch=14-step=375.ckpt' as top 2


Label: tensor([0, 0, 0, 0, 1, 0, 0, 0], device='cuda:0')
Pred: tensor([[ -8.1406],
        [ -8.9453],
        [ -3.8809],
        [-10.1172],
        [ 10.6172],
        [-15.5000],
        [-10.7031],
        [-18.3594]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.150:

Loss: 0.0026165565941482782
Accuracy: 1.0
Label: tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
Pred: tensor([[-10.6172],
        [ -7.4141],
        [ -9.1484],
        [-15.1094],
        [ -4.8047],
        [-11.4766],
        [ -8.4453],
        [ -4.1602]], device='cuda:0', dtype=torch.float16)
>>> Validation step No.151:

Loss: 0.0030751593876630068
Accuracy: 1.0
>>> Epoch end loss: 0.03999999910593033


`Trainer.fit` stopped: `max_epochs=15` reached.


In [37]:
model = VideoModel.load_from_checkpoint(
    checkpoint_path="/home/toghrul/SLR/sign-lang/checkpoints/last-v4.ckpt",
    hparams_file="/home/toghrul/SLR/sign-lang/lightning_logs/version_45/hparams.yaml",
    map_location=None,
)
model = model.cuda()

Using cache found in /home/toghrul/.cache/torch/hub/facebookresearch_pytorchvideo_main


In [97]:
torch.save(model, "../models/eff3d_bin.pt")

In [99]:
eff3d = torch.load("../models/eff3d_bin.pt")
eff3d

VideoModel(
  (video_model): EfficientX3d(
    (s1): Sequential(
      (pathway0_stem_conv_xy): Conv3dTemporalKernel1BnAct(
        (kernel): Sequential(
          (conv): Conv3d(3, 24, kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), bias=False)
          (act): Identity(
            (act): Identity()
          )
        )
      )
      (pathway0_stem_conv): Conv3d5x1x1BnAct(
        (kernel): Sequential(
          (conv): Conv3d(24, 24, kernel_size=(5, 1, 1), stride=(1, 1, 1), padding=(2, 0, 0), groups=24, bias=False)
          (bn): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (act): ReLU(
            (act): ReLU(inplace=True)
          )
        )
      )
    )
    (s2): Sequential(
      (pathway0_res0): X3dBottleneckBlock(
        (_residual_add_func): FloatFunctional(
          (activation_post_process): Identity()
        )
        (_res_proj): Conv3dTemporalKernel1BnAct(
          (kernel): Sequential(
            (conv):

# Testing

In [47]:
test_res = trainer.test(model)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

>> Label: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
>> Pred: [0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0]
              precision    recall  f1-score   support

           0       0.99      1.00      0.99        68
           1       1.00      0.92      0.96        12

    accuracy                           0.99        80
   macro avg       0.99      0.96      0.97        80
weighted avg       0.99      0.99      0.99        80



# Inferencing

In [12]:
from pytorchvideo.data.encoded_video import EncodedVideo

In [16]:
video_path = "../data/binary-data/test/MƏN/2022-04-26 12-58-07.mp4"
video = EncodedVideo.from_path(video_path)

In [46]:
video_data = video.get_clip(0, 2)
video_data = video_transform(video_data)

inputs = video_data["video"].cuda()
inputs = inputs.unsqueeze(0)
inputs.shape

torch.Size([1, 3, 25, 256, 256])

In [13]:
preds = model(inputs).detach().cpu().numpy()
preds = np.where(preds > 0, 1, 0)
bin_idx_to_class[preds[0, 0]]

NameError: name 'model' is not defined

In [14]:
def word_level_prediction(path_to_model, path_to_video):
    model = torch.load(path_to_model)
    video = EncodedVideo.from_path(path_to_video)

    video_data = video.get_clip(0, 2)
    # video_data = video_transform(video_data)

    inputs = video_data["video"].cuda()
    inputs = inputs.unsqueeze(0)
    
    preds = model(inputs).detach().cpu().numpy()
    preds = np.where(preds > 0, 1, 0)
    
    return preds

In [20]:
word_level_prediction("../models/eff3d_bin.pt", video_path)

RuntimeError: CUDA out of memory. Tried to allocate 238.00 MiB (GPU 0; 23.65 GiB total capacity; 18.79 GiB already allocated; 98.19 MiB free; 19.07 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Validation

In [21]:
# val_res = trainer.validate(model)